In [10]:
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns 
import geopandas as gpd
import folium
import warnings
warnings.filterwarnings("ignore")
import re

In [2]:
data_all = pd.read_csv("Iowa_Liquor_Sales.csv")
data_all.head(5)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S29198800001,11/20/2015,2191,Keokuk Spirits,1013 MAIN,KEOKUK,52632,"1013 MAIN\nKEOKUK 52632\n(40.39978, -91.387531)",56.0,Lee,...,297,Templeton Rye w/Flask,6,750,$18.09,$27.14,6,$162.84,4.50,1.19
1,S29195400002,11/21/2015,2205,Ding's Honk And Holler,900 E WASHINGTON,CLARINDA,51632,"900 E WASHINGTON\nCLARINDA 51632\n(40.739238, ...",73.0,Page,...,297,Templeton Rye w/Flask,6,750,$18.09,$27.14,12,$325.68,9.00,2.38
2,S29050300001,11/16/2015,3549,Quicker Liquor Store,1414 48TH ST,FORT MADISON,52627,"1414 48TH ST\nFORT MADISON 52627\n(40.624226, ...",56.0,Lee,...,249,Disaronno Amaretto Cavalli Mignon 3-50ml Pack,20,150,$6.40,$9.60,2,$19.20,0.30,0.08
3,S28867700001,11/04/2015,2513,Hy-Vee Food Store #2 / Iowa City,812 S 1ST AVE,IOWA CITY,52240,812 S 1ST AVE\nIOWA CITY 52240\n,52.0,Johnson,...,237,Knob Creek w/ Crystal Decanter,3,1750,$35.55,$53.34,3,$160.02,5.25,1.39
4,S29050800001,11/17/2015,3942,Twin Town Liquor,104 HIGHWAY 30 WEST,TOLEDO,52342,"104 HIGHWAY 30 WEST\nTOLEDO 52342\n(41.985887,...",86.0,Tama,...,249,Disaronno Amaretto Cavalli Mignon 3-50ml Pack,20,150,$6.40,$9.60,2,$19.20,0.30,0.08


In [3]:
print("The entire dataset dimension: ",data_all.shape) # (12591077, 24)
print("Attributes:",data_all.columns) # (12591077, 24)

The entire dataset dimension:  (12591077, 24)
Attributes: Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)'],
      dtype='object')


In [4]:
# check missing values 
print("Missing values:\n",data_all.isnull().sum()) 

Missing values:
 Invoice/Item Number          0
Date                         0
Store Number                 0
Store Name                   0
Address                   2376
City                      2375
Zip Code                  2420
Store Location            2375
County Number            79178
County                   79178
Category                  8020
Category Name            16086
Vendor Number                3
Vendor Name                  1
Item Number                  0
Item Description             0
Pack                         0
Bottle Volume (ml)           0
State Bottle Cost           10
State Bottle Retail         10
Bottles Sold                 0
Sale (Dollars)              10
Volume Sold (Liters)         0
Volume Sold (Gallons)        0
dtype: int64


In [5]:
# drop records with missing values 
data_all = data_all.dropna()

In [6]:
print("##################\n",data_all.groupby("City").size()) # 790 cities
print("##################\n",data_all.groupby("County").size()) # 200 cities
print("##################\n",data_all.groupby("Item Number").size())# 7263 brands

##################
 City
ACKLEY       2722
ADAIR        1567
ADEL        10878
AFTON         994
AKRON        2094
            ...  
Winthrop      994
Woodbine     2770
Woodward      605
ZWINGLE      1260
Zwingle       455
Length: 790, dtype: int64
##################
 County
ADAIR           2503
ADAMS           2066
ALLAMAKEE      11260
APPANOOSE      10076
AUDUBON         2143
               ...  
Winnebago      33984
Winneshiek     55567
Woodbury      287930
Worth          12643
Wright         27719
Length: 200, dtype: int64
##################
 Item Number
101       31
102       31
104       10
107       31
108       39
          ..
996026     3
997085     1
997086     3
997878     3
999275     1
Length: 7263, dtype: int64


In [7]:
l = data_all.loc[:, ["Volume Sold (Liters)", "Category"]].groupby("Category").sum()
l = l.sort_values(by = "Volume Sold (Liters)", ascending = False).reset_index()
l.iloc[:3, 0].to_list()

[1031080.0, 1012100.0, 1062310.0]

In [8]:
top_three_category = l.iloc[:3, 0].to_list()
data = data_all.loc[data_all["Category"].isin(top_three_category),:]
data.Category.value_counts()
print(data.shape)

(2981686, 24)


### Pre-processing 

In [11]:

data["Date"]  = pd.to_datetime(data['Date'],format='%m/%d/%Y')
data['year'] = pd.DatetimeIndex(data['Date']).year
data['month'] = pd.DatetimeIndex(data['Date']).month
data['day'] = pd.DatetimeIndex(data['Date']).day

#pd.set_option('display.max_rows', 10)
data["County"] = data["County"].str.capitalize()
data["City"] = data["City"].str.capitalize()
county_countN_df = data.loc[:,["County","County Number","Date"]].groupby(["County","County Number"]).count().reset_index()
county_countN_df["County Number"].value_counts()
print(county_countN_df.loc[county_countN_df["County Number"].isin([71.0, 78.0,17.0, 11.0]),:])

data.County = data.County.replace(["buena vist","cerro gord","obrien","pottawatta"], 
                                  ["buena vista","cerro gordo","o'brien","pottawattamie"])


           County  County Number   Date
10     Buena vist           11.0   2807
11    Buena vista           11.0  25146
17     Cerro gord           17.0   6669
18    Cerro gordo           17.0  65819
72        O'brien           71.0  21350
73         Obrien           71.0   3086
80     Pottawatta           78.0   8204
81  Pottawattamie           78.0  84674


In [12]:
# convert data types 
data.loc[:,"Sale (Dollars)"] = data.loc[:, "Sale (Dollars)"].str[1:]
data.loc[:,"State Bottle Cost"] = data.loc[:, "State Bottle Cost"].str[1:]
data.loc[:,"State Bottle Retail"] = data.loc[:, "State Bottle Retail"].str[1:]
data["Store Number"] = data["Store Number"].astype("category")
data["County Number"] = data["County Number"].astype("int")
data["County Number"] = data["County Number"].astype("category")
data["County"] = data["County"].astype("category")
data["City"] = data["City"].astype("category")
data["Sale (Dollars)"] = data["Sale (Dollars)"].astype("float")
data["State Bottle Cost"] = data["State Bottle Cost"].astype("float")
data["State Bottle Retail"] = data["State Bottle Retail"].astype("float")



,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),year,month,day
105,S29065400007,2015-11-16,2555,Hy-Vee Food Store / Keokuk,3111 MAIN,Keokuk,52632,"3111 MAIN\nKEOKUK 52632\n(40.414975, -91.403338)",56,Lee,...,750,8.25,12.38,12,148.56,9.00,2.38,2015,11,16
114,S11716100038,2013-04-17,3990,Cork and Bottle / Oskaloosa,309 A AVE WEST,Oskaloosa,52577,"309 A AVE WEST\nOSKALOOSA 52577\n(41.296228, -...",62,Mahaska,...,750,12.10,18.65,2,37.30,1.50,0.40,2013,4,17
116,S05702000043,2012-05-24,3912,Smokin' Joe's #14 Tobacco and Liquor,225 EDGEWOOD RD,Cedar rapids,52405,225 EDGEWOOD RD\nCEDAR RAPIDS 52405\n(41.97341...,57,Linn,...,750,14.50,21.74,1,21.74,0.75,0.20,2012,5,24
118,S27992700007,2015-09-21,4898,Burlington Shell,"130, S ROOSEVELT AVE",Burlington,52601,"130, S ROOSEVELT AVE\nBURLINGTON 52601\n(40.80...",29,Des moines,...,200,2.75,4.13,10,41.30,2.00,0.53,2015,9,21
122,S19526900004,2014-06-12,4819,Super Stop 2 / Altoona,2635 ADVENTURELAND DR,Altoona,50009,2635 ADVENTURELAND DR\nALTOONA 50009\n(41.6585...,77,Polk,...,1750,6.92,10.38,6,62.28,10.50,2.77,2014,6,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12591042,INV-08368000044,2017-10-31,5423,Stammer Liquor Corp,615 2nd Ave,Sheldon,51201,"615 2nd Ave\nSheldon 51201\n(43.184614, -95.85...",71,Obrien,...,1000,9.71,14.57,1,174.84,1.00,0.26,2017,10,31
12591043,INV-08368000045,2017-10-31,5423,Stammer Liquor Corp,615 2nd Ave,Sheldon,51201,"615 2nd Ave\nSheldon 51201\n(43.184614, -95.85...",71,Obrien,...,1000,6.29,9.44,1,113.28,1.00,0.26,2017,10,31
12591044,INV-08368000046,2017-10-31,5423,Stammer Liquor Corp,615 2nd Ave,Sheldon,51201,"615 2nd Ave\nSheldon 51201\n(43.184614, -95.85...",71,Obrien,...,1750,9.46,14.19,1,85.14,1.75,0.46,2017,10,31
12591045,INV-08368000047,2017-10-31,5423,Stammer Liquor Corp,615 2nd Ave,Sheldon,51201,"615 2nd Ave\nSheldon 51201\n(43.184614, -95.85...",71,Obrien,...,1750,9.97,14.96,2,89.76,3.50,0.92,2017,10,31


In [63]:
data["latitude"]  = [re.findall(r"\((.+?),",i) for i in data["Store Location"]]

#data # 2981046 (about 600 records with missing coordinates)

In [84]:
data["longitude"]  = [re.findall(r", (.+?)\)",i) for i in data["Store Location"]]

In [85]:
data

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),year,month,day,latitude,longitude
105,S29065400007,2015-11-16,2555,Hy-Vee Food Store / Keokuk,3111 MAIN,Keokuk,52632,"3111 MAIN\nKEOKUK 52632\n(40.414975, -91.403338)",56,Lee,...,12.38,12,148.56,9.00,2.38,2015,11,16,[40.414975],[-91.403338]
114,S11716100038,2013-04-17,3990,Cork and Bottle / Oskaloosa,309 A AVE WEST,Oskaloosa,52577,"309 A AVE WEST\nOSKALOOSA 52577\n(41.296228, -...",62,Mahaska,...,18.65,2,37.30,1.50,0.40,2013,4,17,[41.296228],[-92.648153]
116,S05702000043,2012-05-24,3912,Smokin' Joe's #14 Tobacco and Liquor,225 EDGEWOOD RD,Cedar rapids,52405,225 EDGEWOOD RD\nCEDAR RAPIDS 52405\n(41.97341...,57,Linn,...,21.74,1,21.74,0.75,0.20,2012,5,24,[41.973418],[-91.715955]
118,S27992700007,2015-09-21,4898,Burlington Shell,"130, S ROOSEVELT AVE",Burlington,52601,"130, S ROOSEVELT AVE\nBURLINGTON 52601\n(40.80...",29,Des moines,...,4.13,10,41.30,2.00,0.53,2015,9,21,[40.807959],[-91.141422]
122,S19526900004,2014-06-12,4819,Super Stop 2 / Altoona,2635 ADVENTURELAND DR,Altoona,50009,2635 ADVENTURELAND DR\nALTOONA 50009\n(41.6585...,77,Polk,...,10.38,6,62.28,10.50,2.77,2014,6,12,[41.65854],[-93.494642]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12591042,INV-08368000044,2017-10-31,5423,Stammer Liquor Corp,615 2nd Ave,Sheldon,51201,"615 2nd Ave\nSheldon 51201\n(43.184614, -95.85...",71,Obrien,...,14.57,1,174.84,1.00,0.26,2017,10,31,[43.184614],[-95.854743]
12591043,INV-08368000045,2017-10-31,5423,Stammer Liquor Corp,615 2nd Ave,Sheldon,51201,"615 2nd Ave\nSheldon 51201\n(43.184614, -95.85...",71,Obrien,...,9.44,1,113.28,1.00,0.26,2017,10,31,[43.184614],[-95.854743]
12591044,INV-08368000046,2017-10-31,5423,Stammer Liquor Corp,615 2nd Ave,Sheldon,51201,"615 2nd Ave\nSheldon 51201\n(43.184614, -95.85...",71,Obrien,...,14.19,1,85.14,1.75,0.46,2017,10,31,[43.184614],[-95.854743]
12591045,INV-08368000047,2017-10-31,5423,Stammer Liquor Corp,615 2nd Ave,Sheldon,51201,"615 2nd Ave\nSheldon 51201\n(43.184614, -95.85...",71,Obrien,...,14.96,2,89.76,3.50,0.92,2017,10,31,[43.184614],[-95.854743]


In [86]:
data.isnull().sum()

Invoice/Item Number      0
Date                     0
Store Number             0
Store Name               0
Address                  0
City                     0
Zip Code                 0
Store Location           0
County Number            0
County                   0
Category                 0
Category Name            0
Vendor Number            0
Vendor Name              0
Item Number              0
Item Description         0
Pack                     0
Bottle Volume (ml)       0
State Bottle Cost        0
State Bottle Retail      0
Bottles Sold             0
Sale (Dollars)           0
Volume Sold (Liters)     0
Volume Sold (Gallons)    0
year                     0
month                    0
day                      0
latitude                 0
longitude                0
dtype: int64

In [ ]:
c = [re.findall(r"\((.+?),",i) for i in data["Store Location"]]
len(c)

In [60]:
data.iloc[16, 7]

'409 E CENTER\nCONRAD 50621\n'

In [45]:
print(re.findall("r\n(.+?)",data.iloc[1, 7]))

[]


In [36]:
lst= data.iloc[1,7][len(data.iloc[1,7])-22:len(data.iloc[1,7])-1:1].split(",")
[float(i) for i in lst]
#data.iloc[1,7]

[41.296228, -92.648153]

In [14]:
# remove potential errors 
cond1 = data["Bottle Volume (ml)"] == 0
cond2 = data["State Bottle Cost"] == 0
cond3 = data["State Bottle Retail"] == 0
cond4 = data["Sale (Dollars)"] == 0
cond5 = data["Bottles Sold"] ==0
cond6 = data["Volume Sold (Liters)"] ==0
data = data.loc[~(cond1|cond2|cond3|cond4|cond5|cond6),:]
# 640 rows 

In [17]:
data.columns

Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)',
       'year', 'month', 'day'],
      dtype='object')

### Visualisation 

In [46]:
sf = gpd.read_file("county.shp")

In [48]:
sf.head()

,Shape_Leng,Shape_Area,AREA,PERIMETER,CO_NUMBER,CO_FIPS,ACRES,FIPS,COUNTY,ST,geometry
0,192784.420113,1.394780e+09,1.394780e+09,192783.7,56.0,111,344657.643007,19111,Lee,IA,"POLYGON ((634170.676 4519205.254, 634170.989 4..."
1,146566.360164,1.335647e+09,1.335647e+09,146566.5,4.0,7,330045.489981,19007,Appanoose,IA,"POLYGON ((530372.212 4527603.237, 530385.212 4..."
2,147784.689987,1.364467e+09,1.364467e+09,147785.1,93.0,185,337167.259929,19185,Wayne,IA,"POLYGON ((454680.164 4527594.006, 454692.977 4..."
3,148600.585139,1.381359e+09,1.381359e+09,148600.8,27.0,53,341341.146687,19053,Decatur,IA,"POLYGON ((435311.961 4527873.727, 436775.740 4..."
4,144994.231402,1.306355e+09,1.306355e+09,144995.3,26.0,51,322807.376878,19051,Davis,IA,"POLYGON ((569161.195 4526464.814, 569170.313 4..."


In [ ]:
# Analyze total alcohol sales in different counties (chorolength)
data.loc[:, ["County", "Volume Sold (Liters)"]].groupby("County").sum().shape # 103 counties


In [51]:

zone = pd.read_csv("taxi_zone_lookup.csv")

sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
gdf_PUL = gpd.GeoDataFrame(pd.merge(df_filtered, sf, left_on = 'PULocationID', 
                                right_on = 'LocationID')).drop('PULocationID', axis = 1)

geoJSON = gdf_PUL[['LocationID', 'geometry','COUNTY']].drop_duplicates('LocationID').to_json()


FileNotFoundError: [Errno 2] No such file or directory: 'taxi_zone_lookup.csv'

In [53]:
sf.columns

Index(['Shape_Leng', 'Shape_Area', 'AREA', 'PERIMETER', 'CO_NUMBER', 'CO_FIPS',
       'ACRES', 'FIPS', 'COUNTY', 'ST', 'geometry'],
      dtype='object')

In [ ]:
td_map = folium.Map(location = [41.8780, -93.0977], 
                    zoom_start= 10,tiles = "cartodbpositron")

cho_map = folium.Choropleth(
    geo_data=geoJSON,
    data=df_filtered[['PULocationID','unit_fare']],
    name='choropleth',
    fill_color='YlGn',
    fill_opacity=0.7,
    key_on = 'feature.properties.LocationID',
    line_opacity=0.2,
    legend_name='Trip distance',
    highlight=True,
    line_color='green',
    columns = ['PULocationID','unit_fare'], reset = True,
).add_to(td_map)

cho_map.geojson.add_child(folium.features.GeoJsonTooltip(['borough'], 
                                                         labels = False))
folium.LayerControl().add_to(td_map)
td_map.save('foliumChoroplethMap_trip_distance.html')
td_map